In [28]:
import pandas as pd
import re

violations_df = pd.read_csv("./dataset/Traffic_Violations.csv")
violations_df.head()

,SeqID,Date Of Stop,Time Of Stop,Agency,SubAgency,Description,Location,Latitude,Longitude,Accident,...,Charge,Article,Contributed To Accident,Race,Gender,Driver City,Driver State,DL State,Arrest Type,Geolocation
0,52282e8c-f2e1-4bb5-8509-2d5e4f8da8ca,05/01/2023,23:11:00,MCP,"3rd District, Silver Spring",OPERATING UNREGISTERED MOTOR VEHICLE ON HIGHWAY,BRIGGS CHANEY RD @ COLUMIBA PIKE,0.000000,0.00000,No,...,13-401(b1),Transportation Article,False,WHITE,M,GAITHERSBURG,MD,MD,A - Marked Patrol,"(0.0, 0.0)"
1,b66f253b-af29-4bc4-bb73-93755ca2a779,08/31/2023,16:41:00,MCP,"6th District, Gaithersburg / Montgomery Village",DRIVING TO DRIVE MOTOR VEHICLE ON HIGHWAY WITH...,OAKMONT AVE @ GROVEMONT CIR,39.097965,-77.15301,No,...,16-101(a1),Transportation Article,False,HISPANIC,M,GAITHERSBURG,MD,MD,A - Marked Patrol,"(39.097965, -77.15301)"
2,b66f253b-af29-4bc4-bb73-93755ca2a779,08/31/2023,16:41:00,MCP,"6th District, Gaithersburg / Montgomery Village",FAILURE TO DISPLAY REGISTRATION CARD UPON DEMA...,OAKMONT AVE @ GROVEMONT CIR,39.097965,-77.15301,No,...,13-409(b),Transportation Article,False,HISPANIC,M,GAITHERSBURG,MD,MD,A - Marked Patrol,"(39.097965, -77.15301)"
3,b66f253b-af29-4bc4-bb73-93755ca2a779,08/31/2023,16:41:00,MCP,"6th District, Gaithersburg / Montgomery Village",DRIVER OF MOTOR VEHICLE FOLLOWING VEHICLE CLOS...,OAKMONT AVE @ GROVEMONT CIR,39.097965,-77.15301,No,...,21-310(a),Transportation Article,False,HISPANIC,M,GAITHERSBURG,MD,MD,A - Marked Patrol,"(39.097965, -77.15301)"
4,b66f253b-af29-4bc4-bb73-93755ca2a779,08/31/2023,16:41:00,MCP,"6th District, Gaithersburg / Montgomery Village",FAILURE TO CONTROL VEH. SPEED ON HWY. TO AVOID...,OAKMONT AVE @ GROVEMONT CIR,39.097965,-77.15301,No,...,21-801(b),Transportation Article,False,HISPANIC,M,GAITHERSBURG,MD,MD,A - Marked Patrol,"(39.097965, -77.15301)"


In [29]:
violations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2070115 entries, 0 to 2070114
Data columns (total 43 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   SeqID                    object 
 1   Date Of Stop             object 
 2   Time Of Stop             object 
 3   Agency                   object 
 4   SubAgency                object 
 5   Description              object 
 6   Location                 object 
 7   Latitude                 float64
 8   Longitude                float64
 9   Accident                 object 
 10  Belts                    object 
 11  Personal Injury          object 
 12  Property Damage          object 
 13  Fatal                    object 
 14  Commercial License       object 
 15  HAZMAT                   object 
 16  Commercial Vehicle       object 
 17  Alcohol                  object 
 18  Work Zone                object 
 19  Search Conducted         object 
 20  Search Disposition       object 
 21  Search O

<h1><b>SeqID</b></h1>

In [30]:
violations_df = violations_df.drop_duplicates(subset=['SeqID'], keep='first')
violations_df = violations_df.reset_index(drop=True)

violations_df["SeqID"] = violations_df["SeqID"].astype("string")

<h1><b>Date Of Stop</b></h1>

In [31]:
from utils.date_of_stop import clean_date_of_stop

violations_df["Date Of Stop"] = violations_df["Date Of Stop"].apply(clean_date_of_stop)
violations_df.dropna(subset=["Date Of Stop"], inplace=True)

<h1><b>Time Of Stop</b></h1>

In [32]:
from utils.time_of_stop import clean_time_of_stop

violations_df["Time Of Stop"] = violations_df["Time Of Stop"].apply(clean_time_of_stop)
violations_df.dropna(subset=["Time Of Stop"], inplace=True)

<h1><b>Agency</b></h1>

In [33]:
violations_df["Agency"] = violations_df["Agency"].str.upper()

<h1><b>SubAgency</b></h1>

In [ ]:
violations_df["SubAgency"] = violations_df["SubAgency"].apply(lambda x: re.sub(r'\s*/\s*', '/', x))

array(['3rd District, Silver Spring',
       '6th District, Gaithersburg/Montgomery Village',
       '4th District, Wheaton', '5th District, Germantown',
       '2nd District, Bethesda', '1st District, Rockville',
       'Headquarters and Special Operations', 'W15', 'S15'], dtype=object)

<h1><b>Description</b></h1>